In [2]:
# This is code to download and install pytorch
import os
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if os.path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

import torch
print('Version', torch.__version__)
print('CUDA enabled:', torch.cuda.is_available())

    100% |████████████████████████████████| 483.0MB 51.1MB/s 
tcmalloc: large alloc 1073750016 bytes == 0x57c94000 @  0x7f960d32c2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 2.4MB/s 
    100% |████████████████████████████████| 2.0MB 5.7MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
Version 0.4.1
CUDA enabled: True


In [4]:
import os
BASE_PATH = '/gdrive/My Drive/colab_files/fake_review_generator/'
if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)
DATA_PATH = BASE_PATH + 'fake_review_generator/'
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

!pwd
!ls
    
os.chdir(BASE_PATH)
if not os.path.exists(BASE_PATH + 'pt_util.py'):
  !wget https://vinitha910.github.io/pt_util.py
    
os.chdir(DATA_PATH)

if not os.path.exists(DATA_PATH + 'processed_data/Office_Products.csv'):
    !wget https://vinitha910.github.io/office_products_review.tar.gz
    !tar -xvf office_products_review.tar.gz
    !rm office_products_review.tar.gz
os.chdir('/content')

/content
sample_data
--2018-12-07 08:28:42--  https://vinitha910.github.io/pt_util.py
Resolving vinitha910.github.io (vinitha910.github.io)... 185.199.111.153, 185.199.109.153, 185.199.108.153, ...
Connecting to vinitha910.github.io (vinitha910.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6504 (6.4K) [application/octet-stream]
Saving to: ‘pt_util.py’

pt_util.py          100%[===================>]   6.35K  --.-KB/s    in 0s      

2018-12-07 08:28:43 (59.0 MB/s) - ‘pt_util.py’ saved [6504/6504]

--2018-12-07 08:28:44--  https://vinitha910.github.io/office_products_review.tar.gz
Resolving vinitha910.github.io (vinitha910.github.io)... 185.199.111.153, 185.199.109.153, 185.199.108.153, ...
Connecting to vinitha910.github.io (vinitha910.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15176234 (14M) [application/gzip]
Saving to: ‘office_products_review.tar.gz’

office_products_rev 1

In [0]:
import pandas
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import sys
import pickle
import re
sys.path.append(BASE_PATH)
import pt_util
import string
from math import log
from math import exp

In [7]:
from google.colab import drive
drive.mount('/gdrive')
!ls /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
'My Drive'


In [0]:
def prepare_data(data_path, name):
    with open(data_path) as f:
        # This reads all the data from the file, but does not do any processing on it.
        data = f.read()
        data = data.replace(string.whitespace, " ")
        data = data.replace("\n", " ")
        data = data.replace("\t", " ")
        data = data.replace("\x1f", "")
        data = data.replace("\x08", "")
        data = data.replace("\x1c", "")
        
    tokens = []
    data = data[int(0.2*len(data) + 113):int(0.4*len(data) + 113)]
    for character in data:
      tokens.append(character)
    tokens = np.array(tokens)    
    unique_tokens = np.unique(tokens)

    voc2ind = {}
    for i in range(len(unique_tokens)):
      voc2ind[unique_tokens[i]] = i
    
    data_tokens = []
    for char in data:
        data_tokens.append(voc2ind[char])

    ind2voc = {val: key for key, val in voc2ind.items()}

    train_text = data_tokens[:int(0.8*len(data_tokens))]
    test_text = data_tokens[int(0.8*len(data_tokens)):]

    pickle.dump({'tokens': train_text, 'ind2voc': ind2voc, 'voc2ind':voc2ind}, open(DATA_PATH + name + '_chars_train.pkl', 'wb'))
    pickle.dump({'tokens': test_text, 'ind2voc': ind2voc, 'voc2ind':voc2ind}, open(DATA_PATH + name + '_chars_test.pkl', 'wb'))
    
prepare_data(DATA_PATH + 'processed_data/Office_Products.csv', 'office_products')

In [0]:
class Vocabulary(object):
    def __init__(self, data_file):
        with open(data_file, 'rb') as data_file:
            dataset = pickle.load(data_file)
        self.ind2voc = dataset['ind2voc']
        self.voc2ind = dataset['voc2ind']

    # Returns a string representation of the tokens.
    def array_to_words(self, arr):
        return ''.join([self.ind2voc[int(ind)] for ind in arr])

    # Returns a torch tensor representing each token in words.
    def words_to_array(self, words):
        return torch.LongTensor([self.voc2ind[word] for word in words])

    # Returns the size of the vocabulary.
    def __len__(self):
        return len(self.voc2ind)

In [0]:
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, data_file, sequence_length, batch_size):
        super(ReviewsDataset, self).__init__()

        self.sequence_length = sequence_length
        self.batch_size = batch_size
        self.vocab = Vocabulary(data_file)

        with open(data_file, 'rb') as data_pkl:
            dataset = pickle.load(data_pkl)

        self.tokens = dataset['tokens']
        remainder = len(self.tokens) % (self.batch_size*self.sequence_length)
        num_tokens = len(self.tokens) - remainder
        self.tokens = self.tokens[:num_tokens]

        assert len(self.tokens) % batch_size == 0
  
        incr = len(self.tokens)/self.batch_size
        index_range = len(self.tokens)/self.batch_size
        data_start_idx = 0
        label_start_idx = 1
        data_end_idx = data_start_idx + self.sequence_length
        label_end_idx = label_start_idx + self.sequence_length
        batch = 0 
        data = [[]]
        labels = [[]]

        while label_start_idx < len(self.tokens):
            data[batch].append(self.tokens[int(data_start_idx):int(data_end_idx)])
            labels[batch].append(self.tokens[int(label_start_idx):int(label_end_idx)])

            if label_end_idx == index_range:
                data.append([])
                labels.append([])
                data_start_idx = data_end_idx + 1
                label_start_idx = label_end_idx + 1
                data_end_idx = data_start_idx + self.sequence_length
                label_end_idx = label_start_idx + self.sequence_length
                index_range += incr
                batch += 1

            else:
                data_start_idx += self.sequence_length
                label_start_idx += self.sequence_length

                data_end_idx += self.sequence_length
                if data_end_idx > index_range - 1:
                    data_end_idx = index_range - 1;

                label_end_idx += self.sequence_length
                if label_end_idx > index_range:
                    label_end_idx = index_range
        
        self.data = []
        self.labels = []
        for b in range(len(data[0])):
            self.data.append([])
            self.labels.append([])
            for d in range(len(data)):
                if b < len(data[d]):
                    self.data[-1].append(data[d][b])
                    self.labels[-1].append(labels[d][b])
    
        if len(self.data[-1][0]) < self.sequence_length:
            self.data.pop()
            self.labels.pop()
            
    def __len__(self):
        sequences = []
        for batch in self.data:
            for sequence in batch:
                sequences.append(sequence)
        return len((np.array(sequences)))
        
    def __getitem__(self, idx):
        col = int(idx % self.batch_size)
        row = int(idx / self.batch_size)

        if row >= len(self.data) or col >= len(self.data[row]):
            print("ReviewsDataset index out of bounds")
            
        item_data = torch.LongTensor(self.data[row][col])
        item_label = torch.LongTensor(self.labels[row][col])
        
        return item_data, item_label

    def vocab_size(self):
        return len(self.vocab)

In [0]:
class Discriminator(nn.Module):
    def __init__(self, vocab_size, embed_size, feature_size, batch_size, sequence_length):
        super(Discriminator, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.feature_size = feature_size
        self.batch_size = batch_size
        self.encoder = nn.Embedding(self.vocab_size, embed_size)
        self.fully_connected = nn.Linear(self.feature_size*sequence_length, 1)
        self.sig = nn.Sigmoid()
        self.best_accuracy = -1
        
    def forward(self, x, hidden=None):
        # Embed word ids to vectors
        x = self.encoder(x)
        x = x.view(-1,  x.size()[1]*x.size()[2]) 
        x = self.fully_connected(x)
        x = self.sig(x)
        return x
      
    def loss(self, prediction, label, reduction='elementwise_mean'):
        loss_val = F.binary_cross_entropy(prediction, label, reduction=reduction)
        return loss_val
      
    # Saves the current model
    def save_model(self, file_path, num_to_keep=1):
        pt_util.save(self, file_path, num_to_keep)

    # Saves the best model so far
    def save_best_model(self, accuracy, file_path, num_to_keep=1):
        if accuracy > self.best_accuracy:
            self.save_model(file_path, num_to_keep)
            self.best_accuracy = accuracy

    def load_model(self, file_path):
        pt_util.restore(self, file_path)

    def load_last_model(self, dir_path):
        return pt_util.restore_latest(self, dir_path)

In [0]:
class Generator(nn.Module):
    def __init__(self, vocab_size, embed_size, feature_size, num_layers):
        super(Generator, self).__init__()
        self.vocab_size = vocab_size
        self.num_layers = num_layers
        self.encoder = nn.Embedding(self.vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, feature_size, num_layers, batch_first=True)
        self.decoder = nn.Linear(feature_size, self.vocab_size)
        
        self.decoder.weight = self.encoder.weight
        self.decoder.bias.data.zero_()
        
        self.best_accuracy = -1
        
    def forward(self, x, hidden=None):
        # Embed word ids to vectors
        x = self.encoder(x)
         
        # Forward propagate LSTM
        output, (hidden, c) = self.lstm(x, hidden)
        
        # Reshape output to (batch_size*sequence_length, feature_size)
        output = output.reshape(output.size(0)*output.size(1), output.size(2))
        
        # Decode hidden states of all time steps
        output = self.decoder(output)
        return output, (hidden, c)
      
    # This defines the function that gives a probability distribution and implements the temperature computation.
    def inference(self, x, hidden_state=None, temperature=1.5):
        x = x.view(-1, 1)
        x, hidden_state = self.forward(x, hidden_state)
        x = x.view(1, -1)
        x = x / max(temperature, 1e-20)
        x = F.softmax(x, dim=1)
        return x, hidden_state
      
    def loss(self, prediction, label, reduction='elementwise_mean'):
        loss_val = F.cross_entropy(prediction.view(-1, self.vocab_size), label.view(-1), reduction=reduction)
        return loss_val
      
    # Saves the current model
    def save_model(self, file_path, num_to_keep=1):
        pt_util.save(self, file_path, num_to_keep)

    # Saves the best model so far
    def save_best_model(self, accuracy, file_path, num_to_keep=1):
        if accuracy > self.best_accuracy:
            self.save_model(file_path, num_to_keep)
            self.best_accuracy = accuracy

    def load_model(self, file_path):
        pt_util.restore(self, file_path)

    def load_last_model(self, dir_path):
        return pt_util.restore_latest(self, dir_path)

In [0]:

BEAM_WIDTH = 10

def generate_language(model, device, seed_words, sequence_length, vocab, beam_width=BEAM_WIDTH, use_indices=False, sampling_strategy='sample'):
    model.eval()

    with torch.no_grad():  
        seed_words_arr = vocab.words_to_array(seed_words)
        
        # Computes the initial hidden state from the prompt (seed words).
        hidden = None
        for ind in seed_words_arr:
            data = ind.to(device)
            output, hidden = model.inference(data, hidden)

        outputs = []
        # Initializes the beam list.
        beams = [([], output, hidden, 0)]
        
        for ii in range(sequence_length):
            if sampling_strategy == 'sample':
                probabilities = output.exp()
                val = torch.multinomial(probabilities, num_samples=1)
                outputs += [val[0]]
                output, hidden = model.inference(val[0], hidden)
                
            elif sampling_strategy == 'beam':
                all_beams = list()
                # For each beam in the beam list
                for i in range(len(beams)):
                    sequence, output, hidden, score = beams[i]

                    if (len(sequence) > 0):
                        # Compute the next distribution over the output space for that state
                        output, hidden = model.inference(sequence[-1], hidden)

                    # Sample from the distribution    
                    samples = torch.multinomial(output, BEAM_WIDTH)

                    # For each sample
                    for sample in samples[0]:
                        # Compute its score and Record its hidden state and chosen value
                        beam = (sequence + [sample], output, hidden, score + log(output[0][sample]))
                        # Add all the samples to the new beam list
                        all_beams.append(beam)

                # Rank the new beam list
                ordered_beams = sorted(all_beams, key=lambda beam:beam[3], reverse=True)

                # Throw out all but the top N beams
                beams = ordered_beams[:5]

                # Return the top beam's chosen values
                outputs = beams[0][0]
        
        if not use_indices:
            return vocab.array_to_words(seed_words_arr.tolist() + outputs)
        else:
            return seed_words_arr.tolist() + outputs

In [0]:
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

def train(generator, discriminator, device, train_loader, lr, epoch, log_interval, vocab, batch_size, sequence_length):
    losses = []
    hidden = None
    for batch_idx, (data, label) in enumerate(train_loader):
        human_data, label = data.to(device), label.to(device)
        # Separates the hidden state across batches. 
        # Otherwise the backward would try to go all the way to the beginning every time.
        if hidden is not None:
            hidden = repackage_hidden(hidden)
        
        human_labels = torch.ones(BATCH_SIZE, 1).to(device)
        machine_labels = torch.zeros(BATCH_SIZE, 1).to(device)
        
        # Discriminator training: detect machine input
        d_output = discriminator(human_data)
        human_loss = discriminator.loss(d_output, human_labels)
        
        fake_data = []
        for batch in range(batch_size):
          seed_words_id = torch.multinomial(torch.ones(len(vocab)), num_samples=1).unsqueeze(1)
          seed_word = vocab.array_to_words([seed_words_id.item()])
          fake_sentence = generate_language(generator, device, seed_word[0], sequence_length - 1, vocab, beam_width=15, use_indices=True)
          fake_data.append(fake_sentence)
        fake_data = torch.from_numpy(np.array(fake_data)).to(device)
        
        discriminator.train()
        d_output = discriminator(fake_data)
        machine_loss = discriminator.loss(d_output, machine_labels)
        
        discriminator_loss = human_loss + machine_loss
        discriminator_optimizer.zero_grad()
        generator_optimizer.zero_grad()
        discriminator_loss.backward()
        discriminator_optimizer.step()
        
        
        # Generator training
        # Create a different set of data
        fake_data = []
        for batch in range(batch_size):
          seed_words_id = torch.multinomial(torch.ones(len(vocab)), num_samples=1).unsqueeze(1)
          seed_word = vocab.array_to_words([seed_words_id.item()])
          fake_sentence = generate_language(generator, device, seed_word[0], sequence_length - 1, vocab, beam_width=15, use_indices=True)
          fake_data.append(fake_sentence)
        fake_data = torch.from_numpy(np.array(fake_data)).to(device)

        # Tell discriminator the fake data is real
        # How well does discriminator being tricked?
        d_output = discriminator(fake_data)
        
        # Calculate the loss for the discriminator to distinguish between real/fake launguage
        trickery_loss = discriminator.loss(d_output, human_labels)
 
        # Calculate the loss for the generators ability to produce meaningful language
        generator_optimizer.zero_grad()
        generator.train()
        output, hidden = generator(human_data, hidden)
        pred = output.max(-1)[1]
        language_loss = generator.loss(output, label)
        
        # The loss for the generator is the sum of the loss for tricking and language generation
        generator_loss = trickery_loss + language_loss
        
        losses.append(generator_loss.item())
        generator.train()
        generator_loss.backward()
        generator_optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLanguage Loss: {:.6f}\tTrickery Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), language_loss.item(), trickery_loss.item()))
            generator.save_model(DATA_PATH + 'checkpoints/%03d.pt' % epoch, 0)
    return np.mean(losses)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        hidden = None
        for batch_idx, (data, label) in enumerate(test_loader):
            data, label = data.to(device), label.to(device)
            output, hidden = model(data, hidden)
            test_loss += model.loss(output, label).item()
            pred = output.max(-1)[1]
            correct_mask = pred.eq(label.view_as(pred))
            num_correct = correct_mask.sum().item()
            correct += num_correct

    test_loss /= len(test_loader)
    test_accuracy = 100. * correct / (len(test_loader.dataset) * test_loader.dataset.sequence_length)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset) * test_loader.dataset.sequence_length,
        100. * correct / (len(test_loader.dataset) * test_loader.dataset.sequence_length)))
    return test_loss, test_accuracy

In [0]:
SEQUENCE_LENGTH = 50
BATCH_SIZE = 32
EMBED_SIZE = 128
FEATURE_SIZE = 128
TEST_BATCH_SIZE = 16
EPOCHS = 30
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0005
USE_CUDA = True
PRINT_INTERVAL = 10
LOG_PATH = DATA_PATH + 'logs/log.pkl'
NUM_LAYERS = 1
!export CUDA_LAUNCH_BLOCKING=1; 

data_train = ReviewsDataset(DATA_PATH + 'office_products_chars_train.pkl', SEQUENCE_LENGTH, BATCH_SIZE)
data_test = ReviewsDataset(DATA_PATH + 'office_products_chars_test.pkl', SEQUENCE_LENGTH, TEST_BATCH_SIZE)
vocab = data_train.vocab

use_cuda = USE_CUDA and torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")
print('Using device', device)
import multiprocessing
num_workers = multiprocessing.cpu_count()
print('num workers:', num_workers)

kwargs = {'num_workers': num_workers,
          'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(data_train, batch_size=BATCH_SIZE,
                                           shuffle=False, **kwargs)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=TEST_BATCH_SIZE,
                                          shuffle=False, **kwargs)

generator = Generator(data_train.vocab_size(), EMBED_SIZE, FEATURE_SIZE, NUM_LAYERS).to(device)
discriminator = Discriminator(data_train.vocab_size(), EMBED_SIZE, FEATURE_SIZE, BATCH_SIZE, SEQUENCE_LENGTH).to(device)

generator_optimizer = optim.Adam(generator.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

start_epoch = generator.load_last_model(DATA_PATH + 'checkpoints')

train_losses, test_losses, test_accuracies, train_p, test_p = pt_util.read_log(LOG_PATH, ([], [], [], [], []))
test_loss, test_accuracy = test(generator, device, test_loader)

test_losses.append((start_epoch, test_loss))
test_accuracies.append((start_epoch, test_accuracy))

try:
    for epoch in range(start_epoch, EPOCHS + 1):
        lr = LEARNING_RATE * np.power(0.25, (int(epoch / 6)))
        train_loss = train(generator, discriminator, device, train_loader, lr, epoch, PRINT_INTERVAL, vocab, BATCH_SIZE, SEQUENCE_LENGTH)
        test_loss, test_accuracy = test(generator, device, test_loader)
        train_losses.append((epoch, train_loss))
        test_losses.append((epoch, test_loss))
        test_accuracies.append((epoch, test_accuracy))
        test_p.append((epoch, exp(test_loss)))
        train_p.append((epoch, exp(train_loss)))
        pt_util.write_log(LOG_PATH, (train_losses, test_losses, test_accuracies, train_p, test_p))
        generator.save_best_model(test_accuracy, DATA_PATH + 'checkpoints/%03d.pt' % epoch)
        seed_words = 'This printer quality is'
        generated_sentence = generate_language(generator, device, seed_words, 200, vocab)
        print('generated beam\t\t', generated_sentence)
        print('')

except KeyboardInterrupt as ke:
    print('Interrupted')
except:
    import traceback
    traceback.print_exc()
finally:
    print('Saving final model')
    generator.save_model(DATA_PATH + 'checkpoints/%03d.pt' % epoch, 0)

Using device cuda
num workers: 2

Test set: Average loss: 5.8966, Accuracy: 43511/1708000 (3%)

Train Epoch: 0 [0/136672 (0%)]	Language Loss: 5.911780	Trickery Loss: 0.844135
Saved /gdrive/My Drive/colab_files/fake_review_generator/fake_review_generator/checkpoints/000.pt

Train Epoch: 0 [320/136672 (0%)]	Language Loss: 3.082562	Trickery Loss: 1.845164
Saved /gdrive/My Drive/colab_files/fake_review_generator/fake_review_generator/checkpoints/000.pt

Train Epoch: 0 [640/136672 (0%)]	Language Loss: 2.637703	Trickery Loss: 3.493129
Saved /gdrive/My Drive/colab_files/fake_review_generator/fake_review_generator/checkpoints/000.pt

Train Epoch: 0 [960/136672 (1%)]	Language Loss: 2.537626	Trickery Loss: 4.567259
Saved /gdrive/My Drive/colab_files/fake_review_generator/fake_review_generator/checkpoints/000.pt

Train Epoch: 0 [1280/136672 (1%)]	Language Loss: 2.401195	Trickery Loss: 5.055908
Saved /gdrive/My Drive/colab_files/fake_review_generator/fake_review_generator/checkpoints/000.pt

Train

In [0]:
seed_words = 'This printer is'
sequence_length = 50

for ii in range(10):
    generated_sentence = generate_language(generator, device, seed_words, sequence_length, vocab, 'beam')
    print('generated with beam\t', generated_sentence)


generated with beam	 This printer isx7!;qMkdr!m2id7vT.'z/zOy&n_>|g:'3oD*diQi2b!5WN7Fqe
generated with beam	 This printer isDD=]g[e5r#L=Lnt)")3B|G}68Ua$mY:6]02Ec}$MtNlG.aY[:_
generated with beam	 This printer isX*Y^+.dJ+xc)yJ%Ot$9-z,1>v+nYu*Ff6%E)Tpbz(QZ"6.}M'J
generated with beam	 This printer isg#?ON%w[*V/dG3LS@]-9>>r7nqs-7*cCE 4cDi `:r]ZbLe"HJ
generated with beam	 This printer is.;-b2/7,,9+(@V8^zLUrG|EUxE=/npBKP3naN?NhoIu^KByq^)
generated with beam	 This printer is/Z56"xCS[GIeYiVn!i6S>:a^g.KIiBV+6Hd}v,bph!Z~fu2zEr
generated with beam	 This printer isk!H&wn60q4Nw)XJy!qo? e-a19S ]0x9*h,  5p2,tD[y[Iy73
generated with beam	 This printer is}W_FL%XF!(6LMgLk)!0woep1?KZy*6fT3Y-/'k6aS=P6prW.|5
generated with beam	 This printer isQ0&4q(a'FRdhSrvC.4zqa7u/|-[:8=/y22^-+/#MJyQ>pUPwGT
generated with beam	 This printer is-";ajB.=uWJFBAAdaDAlS&3Z}-xKO=/&qO,THTK;UFV,e/,%u 
